### 위키피디아 한국어 Corpus에 Word2Vec 알고리즘 적용한 wiki.model 생성 및 이를 활용한 유사어 조사 

In [ ]:
# 작업시간 유의(코퍼스 생성하기, wiki.txt, wiki.wakati 파일 복사 활용)
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from gensim.models import word2vec
# 파일 열기
readFp = codecs.open("./dataset/wiki.txt", "r", encoding="utf-8")
wakati_file = "wiki.wakati"
writeFp = open(wakati_file, "w", encoding="utf-8")

# 형태소 분석 
okt = Okt()
i = 0
# 텍스트를 한 줄씩 처리하기
while True:
    line = readFp.readline()
    if not line: break
    if i % 20000 == 0:
        print("current - " + str(i))
    i += 1
    # 형태소 분석
    malist = okt.pos(line, norm=True, stem=True)
    # 필요한 어구만 대상으로 하기
    r = []
    for word in malist:
        # 어미/조사/구두점 등은 대상에서 제외 
        if not word[1] in ["Josa", "Eomi", "Punctuation"]:
            writeFp.write(word[0] + " ")
writeFp.close()


In [ ]:
# 작업시간 유의(Word2Vec skip-gram 알고리즘을 사용한 모델 생성 및 저장)
from gensim.models import word2vec
# 띄어쓰기로 구분된 파일을 읽어 들여 코퍼스로 사용할 수 있게 분할
data = word2vec.Text8Corpus("./dataset/wiki.wakati")
# 모델을 생성
# 매개변수 : sg 알고리즘 선택(1=Skip-gram, 0=CBOW), size 벡터의 차원 설정, 
# window 학습할 단어를 연관시킬 앞뒤의 단어 수
model = word2vec.Word2Vec(data, sg=1, size=100, window=5)
# 생성한 모델을 파일로 저장
model.save("./dataset/wiki.model")
print("ok")

In [ ]:
# Gensim 설치 : 자연 언어 처리를 위한 라이브러리로 Word2Vec 기능 포함
# !pip install gensim

In [2]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('./dataset/wiki.model')

In [5]:
# 위키피디아 모델 파일 활용, 유사어 조사
md1 = model.wv.most_similar(positive=['NLP','자연어'])
print(len(md1))
print(md1)

10
[('전산언어학', 0.7927480936050415), ('객체지향', 0.774531364440918), ('자연언어', 0.7666630744934082), ('말뭉치', 0.7620400786399841), ('UML', 0.7490293383598328), ('통사론', 0.7273504734039307), ('전문용어', 0.7267319560050964), ('전산학', 0.7123903036117554), ('휴리스틱', 0.7028846740722656), ('기계번역', 0.6998603940010071)]


In [6]:
# '아빠 - 남성 + 여성' 계산
model.wv.most_similar(positive=['아빠','여성'], negative=['남성'])[0]

('엄마', 0.8511905074119568)

In [4]:
# '왕자 - 남성 + 여성'
model.wv.most_similar(positive=['왕자','여성'],negative=['남성'])[0:10]

[('왕녀', 0.7457159757614136),
 ('여왕', 0.6352607011795044),
 ('왕', 0.619939923286438),
 ('대왕', 0.5986748933792114),
 ('삼촌', 0.5951403379440308),
 ('왕세자', 0.5889936685562134),
 ('왕비', 0.5851935744285583),
 ('갈라드리엘', 0.5823298096656799),
 ('공주', 0.5798119902610779),
 ('아들', 0.5790980458259583)]

In [7]:
# 한국에서 서울에 해당하는 곳은 일본에서 어디인가요? '서울 - 한국 + 일본'
model.wv.most_similar(positive=['서울','일본'],negative=['한국'])[0:5]

[('도쿄', 0.676599383354187),
 ('오사카', 0.6547168493270874),
 ('교토', 0.6285207867622375),
 ('서울특별시', 0.5485527515411377),
 ('후쿠오카', 0.5434316992759705)]

In [6]:
# 중국의 수도
model.wv.most_similar(positive=['서울','중국'],negative=['한국'])[0:5]

[('베이징', 0.6575207114219666),
 ('북경', 0.6304327845573425),
 ('봉천', 0.6280224323272705),
 ('항주', 0.6195797920227051),
 ('남경', 0.6111494302749634)]

In [7]:
# 오른쪽과 왼쪽이라는 반대되는 개념을 넣고 한쪽에 남자를 넣으면?
model.wv.most_similar(positive=['오른쪽','남자'],negative=['왼쪽'])[0]

('여자', 0.780625581741333)

In [8]:
# 서울에서 맛집으로 유명한 지역은 어디일까요?
model.wv.most_similar(positive=['서울','맛집'])[0:5]

[('강남', 0.6651614904403687),
 ('여의도', 0.6196600794792175),
 ('인사동', 0.6163415908813477),
 ('연희동', 0.6118510365486145),
 ('서울특별시', 0.6044690012931824)]

In [9]:
# 벡터데이터 확인 : 100차원 벡터 데이터(size = 100)
print((model.wv['고양이'])[0:5])
print(len(model.wv['고양이']))
print(len(model.wv['강아지']))

[-1.0280149  -0.5838621  -0.4873174  -0.21167026 -1.8950337 ]
100
100


In [10]:
# 모델을 사용해 계산하기 - 유의어
from gensim.models import word2vec
model = word2vec.Word2Vec.load("./dataset/wiki.model")
results = model.wv.most_similar(positive=['과자'])
for result in results:
    print(result)

('케이크', 0.8118672370910645)
('주먹밥', 0.8109839558601379)
('디저트', 0.8098270297050476)
('튀김', 0.8025590181350708)
('김밥', 0.7943276762962341)
('돈가스', 0.7917213439941406)
('비스킷', 0.7907963991165161)
('야채', 0.7853391170501709)
('닭고기', 0.7846125960350037)
('조미료', 0.7839272618293762)


In [13]:
# 지원 센터에서 업무 개선을 위해 메일과 채팅으로 접수된 문장이 
# 급한 내용일 경우 우선도를 올려 대응해야 하는 경우
# 메일에 포함된 문장을 형태소 분석하고 Word2Vec로 '급하다'와
# 유사도가 어느 정도인지 확인하는 방법으로 변경
from konlpy.tag import Okt
from gensim.models import word2vec
# Word2Vec 모델 읽어 들이고 형태소 분석 준비하기
model = word2vec.Word2Vec.load("./dataset/wiki.model") 
okt = Okt()
def print_emargency(text):
    print(text)
    # 전달된 문장을 형태소 분석하기
    node = okt.pos(text, norm=True, stem=True)
    for word, form in node:    
    # 필요한 형태소만 추출하기
        if form == 'Noun' or form == 'Verb' or form == 'Adjective' or form == 'Adverb':            
      # 급하다와 비슷한 단어
            print("-", word, ":", model.wv.similarity(word, '급하다'))
# 첫 문장은 갑자기 빨리, 요청 등 전체적으로 높은 값이 나온 반면 두번째 문장은 비교적 낮은 값이 나옴
print_emargency("컴퓨터에 갑자기 문제가 생겼어요. 빨리 처리할 문제가 있어서 해결 요청합니다.")
print_emargency("사용 방법을 문의 드립니다.")

컴퓨터에 갑자기 문제가 생겼어요. 빨리 처리할 문제가 있어서 해결 요청합니다.
- 컴퓨터 : -0.07506699
- 갑자기 : 0.5643145
- 문제 : 0.25628287
- 생기다 : 0.2879802
- 빨리 : 0.5591332
- 처리 : 0.18828717
- 하다 : 0.42797402
- 문제 : 0.25628287
- 있다 : 0.31476456
- 해결 : 0.19012733
- 요청 : 0.35598242
- 하다 : 0.42797402
사용 방법을 문의 드립니다.
- 사용 : -0.048614163
- 방법 : 0.25084788
- 문의 : 0.27857858
- 드리다 : 0.25129128
